## Getting historical data from CryptoCompare API

In [2]:
import requests
from time import sleep
from json import dumps, loads
from datetime import datetime
from funcy import project

In [131]:
def transform_time(time):
    '''
    Transforms time from timestamp format into day-month-year hour-minute-seconds format
    '''
    fmt = "%d-%m-%Y %H:%M:%S"
    
    # local time
    t = datetime.fromtimestamp(float(time))
    
    return t.strftime(fmt)

In [130]:
def crypto_data():
    '''
    Returns list of dictionaries with price and timestamp of Bitcoin for the last 7 days
    '''
    coin = 'BTC'
    url = "https://min-api.cryptocompare.com/data/v2/histominute?fsym=BTC&tsym=EUR&limit=2000"
    
    dataset = []
    
    for i in range(0,10):
        # Calls API to retrieve 2,000 records of historical BTC data (per minute) 
        records = loads(requests.get(url).text)
        
        if records['Response'] == 'Success':
            # Earliest timestamp in the 2,000 records retrieved
            earliest_timestamp = records['Data']['TimeFrom']

            # List of dictionaries with the crypto data
            crypto_data = records['Data']['Data'][1:]

            # Add newly retrieved crypto_data to existing dataset
            dataset = crypto_data + dataset

            url = "https://min-api.cryptocompare.com/data/v2/histominute?fsym=BTC&tsym=EUR&limit=2000&toTs={}".format(earliest_timestamp)
    
    # Removes all unnessary keys from dictionaries (only keeps time and open price)
    dataset = [project(dictionary, ['time', 'open']) for dictionary in dataset]
    
    # Transforms time into timestamp
    [dictionary.update({'time':transform_time(dictionary['time'])}) for dictionary in dataset]
    
    return dataset

In [129]:
crypto_data[0:5]

[{'time': '22-10-2019 21:22:00', 'open': 7374.34},
 {'time': '22-10-2019 21:23:00', 'open': 7372.7},
 {'time': '22-10-2019 21:24:00', 'open': 7375.12},
 {'time': '22-10-2019 21:25:00', 'open': 7374.18},
 {'time': '22-10-2019 21:26:00', 'open': 7374.47}]